In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import itertools

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    read_loinc_df,
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_siteid_country_map,
    get_siteid_color_maps,
    read_full_demographics_df,
    get_visualization_subtitle,
    get_country_color_map,
    apply_theme,
    merge_single_site_country_name,
)
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
DATA_RELEASE = "2020-09-01"
COHORT = "Adult"
MERGE_SINGLE_SITE_COUNTRIES = False

In [ ]:
df = read_full_demographics_df()
df.head();

## Remove pediatric sites

In [ ]:
df = df.loc[df["pediatric"] == False]

In [ ]:
def flatten(list_of_lists):
    "Flatten one level of nesting"
    return list(itertools.chain.from_iterable(list_of_lists))

In [ ]:
# Replace missing values with np.nan
df = df.replace(-99, np.nan)
df = df.replace(-999, np.nan)

In [ ]:
# Some race categories may have zero patients after summing across all sites.
zero_patients_by_race_df = (df.groupby("race").sum()["num_patients_all"] == 0).to_frame()
races_with_zero_patients = zero_patients_by_race_df.loc[zero_patients_by_race_df["num_patients_all"] == True].index.values.tolist()
df = df.loc[~df["race"].isin(races_with_zero_patients)]
df.head();

In [ ]:
# Human-readable mappings
HR_COLNAME_MAP = {
    "sex": "Sex",
    "race": "Race",
    "age_group": "Age Group",
    "country": "Country"
}

HR_SEVERITY_MAP = {
    "all": "All",
    "ever_severe": "Ever Severe",
    "never_severe": "Never Severe"
}
HR_SEX_MAP = {
    "all": "All",
    "male": "Male",
    "female": "Female",
    "other": "Other",
}
HR_AGE_GROUP_MAP = {
    "all": "All",
    "18to25": "18 - 25",
    "26to49": "26 - 49",
    "50to69": "50 - 69",
    "70to79": "70 - 79",
    "80plus": "80 +",
    "other": "Other",
}
HR_RACE_MAP = {
    'all': 'All',
    'american_indian': 'Amer. Indian or Alaska Nat.',
    'asian': 'Asian',
    'black': 'Black',
    'hawaiian_pacific_islander': 'Nat. Hawaiian or Other Pac. Isl.',
    'hispanic_latino': 'Hispanic or Latino',
    'white': 'White',
    'other': 'Other',
}
HR_SEVERITY_COLNAME_MAP = {
    "num_patients_all": "All",
    "num_patients_ever_severe": "Ever Severe",
    "num_patients_never_severe": "Never Severe"
}

SEVERITY_VALUES = list(HR_SEVERITY_MAP.values())
SEVERITY_VALUES_NA = SEVERITY_VALUES[1:] # NA = "No ALL"
SEX_VALUES = list(HR_SEX_MAP.values())
SEX_VALUES_NA = SEX_VALUES[1:] # NA = "No ALL"
AGE_GROUP_VALUES = list(HR_AGE_GROUP_MAP.values())
AGE_GROUP_VALUES_NA = AGE_GROUP_VALUES[1:] # NA = "No ALL"
RACE_VALUES = list(HR_RACE_MAP.values())
RACE_VALUES_NA = RACE_VALUES[1:] # NA = "No ALL"
COUNTRY_VALUES = df["country"].unique().tolist()

In [ ]:
df["sex"] = df["sex"].replace(HR_SEX_MAP)
df["race"] = df["race"].replace(HR_RACE_MAP)
df["age_group"] = df["age_group"].replace(HR_AGE_GROUP_MAP)

In [ ]:
df = df.loc[df["age_group"].isin(AGE_GROUP_VALUES)]
df.head()

In [ ]:
NUM_SITES = len(df["siteid"].unique().tolist())

In [ ]:
# Make a list of the possible roll-up rows, for the 3 columns (age_group, race, sex)
# Here, None is a wildcard.
partial_roll_ups = list(itertools.product(["All", None], repeat=3))
partial_roll_ups.pop(); # pop off (None, None, None)

In [ ]:
# Fill in the None wildcards with values,
# expanding the list to the real expected number of rows.
more_partial_roll_ups = []
for (age_val, race_val, sex_val) in partial_roll_ups:
    if age_val == None:
        age_vals = AGE_GROUP_VALUES_NA
    else:
        age_vals = [age_val]
    if race_val == None:
        race_vals = RACE_VALUES_NA
    else:
        race_vals = [race_val]
    if sex_val == None:
        sex_vals = SEX_VALUES_NA
    else:
        sex_vals = [sex_val]
    more = list(itertools.product(age_vals, race_vals, sex_vals))
    more_partial_roll_ups.append(more)
more_partial_roll_ups = flatten(more_partial_roll_ups)

In [ ]:
# Make a dict of tuples to values: convert "All" to the list of individual values.
mpru_to_values = dict()
for (age_val, race_val, sex_val) in more_partial_roll_ups:
    if age_val == "All":
        age_vals = AGE_GROUP_VALUES_NA
    else:
        age_vals = [age_val]
    if race_val == "All":
        race_vals = RACE_VALUES_NA
    else:
        race_vals = [race_val]
    if sex_val == "All":
        sex_vals = SEX_VALUES_NA
    else:
        sex_vals = [sex_val]
    
    mpru_to_values[(age_val, race_val, sex_val)] = (age_vals, race_vals, sex_vals)

In [ ]:
expected_rows = list(itertools.product(AGE_GROUP_VALUES_NA, RACE_VALUES_NA, SEX_VALUES_NA))

In [ ]:
# Check that all sites provide the expected non-rolled-up rows.

nonexistant_expected_rows = []

for site_id, site_df in df.groupby("siteid"):
    for (age_val, race_val, sex_val) in expected_rows:
        row_exists = site_df.loc[(
            (site_df["age_group"] == age_val)
            & (site_df["race"] == race_val)
            & (site_df["sex"] == sex_val)
        )].shape[0] > 0
        
        if not row_exists:
            nonexistant_expected_rows.append((site_id, (age_val, race_val, sex_val)))

In [ ]:
# Fill in the expected but missing rows with NaN

siteid_to_country = get_siteid_country_map()
siteid_to_color = get_siteid_color_maps()
    
for (site_id, (age_val, race_val, sex_val)) in nonexistant_expected_rows:
    df = df.append({
        "siteid": site_id,
        "sex": sex_val,
        "age_group": age_val,
        "race": race_val,
        "num_patients_all": np.nan,
        "num_patients_ever_severe": np.nan,
        "country": siteid_to_country[site_id],
        "color": siteid_to_color[site_id],
    }, ignore_index=True)

In [ ]:
# Not all sites provide the partially-"rolled-up" classes.
# Some provide (All, All, All) + (All, y, z) + (x, All, y) + (x, y, All) + (All, All, z) + (All, y, All) + (x, All, All)
# while others only provide (All, All, All) without the rest.

# Here, we can try to create those additional rows
for site_id, site_df in df.groupby("siteid"):
    
    for (age_val, race_val, sex_val) in more_partial_roll_ups:
        row_exists = site_df.loc[(
            (site_df["age_group"] == age_val)
            & (site_df["race"] == race_val)
            & (site_df["sex"] == sex_val)
        )].shape[0] > 0
        
        if not row_exists:
            (age_vals, race_vals, sex_vals) = mpru_to_values[(age_val, race_val, sex_val)]
            roll_up_df = site_df.loc[(
                (site_df["age_group"].isin(age_vals))
                & (site_df["race"].isin(race_vals))
                & (site_df["sex"].isin(sex_vals))
            )]
            roll_up_series = roll_up_df.sum()
            num_patients_all = roll_up_series["num_patients_all"]
            num_patients_ever_severe = roll_up_series["num_patients_ever_severe"]
            
            df = df.append({
                "siteid": site_id,
                "sex": sex_val,
                "age_group": age_val,
                "race": race_val,
                "num_patients_all": num_patients_all,
                "num_patients_ever_severe": num_patients_ever_severe,
                "country": siteid_to_country[site_id],
                "color": siteid_to_color[site_id],
            }, ignore_index=True)

In [ ]:
df["num_patients_never_severe"] = df["num_patients_all"] - df["num_patients_ever_severe"]

In [ ]:
overall_df = df.groupby(["sex", "race", "age_group"]).sum().reset_index()
overall_df.head();

In [ ]:
if MERGE_SINGLE_SITE_COUNTRIES:
    df["country"] = df["country"].apply(merge_single_site_country_name)

country_df = df.groupby(["country", "color", "sex", "race", "age_group"]).sum().reset_index()
COUNTRY_VALUES = df["country"].unique().tolist()

In [ ]:
country_color_map = get_country_color_map(merge_single_site_countries=MERGE_SINGLE_SITE_COUNTRIES)

In [ ]:
def compute_ci(df, n_col="num_patients_ever_severe", N_col="num_patients_all"):
    df["p"] = df[n_col] / df[N_col]

    def calculate_ci(row, upper=False):
        try:
            return row["p"] + ((1 if upper else -1) * (1.96 * np.sqrt((row["p"]*(1 - row["p"])) / row[N_col])))
        except ZeroDivisionError:
            return np.nan

    df["ci_95_lower"] = df.apply(lambda row: calculate_ci(row, upper=False), axis=1)
    df["ci_95_upper"] = df.apply(lambda row: calculate_ci(row, upper=True), axis=1)

    df["np"] = df[N_col] * df["p"]
    df["n1minusp"] = df[N_col] * (1 - df["p"])

    df["p_x100"] = df["p"] * 100
    df["ci_95_lower_x100"] = df["ci_95_lower"] * 100
    df["ci_95_upper_x100"] = df["ci_95_upper"] * 100
    
    
    # Clip values < 0 and 100 >
    df["p_x100"] = df["p_x100"].clip(lower=0, upper=100)
    df["ci_95_lower_x100"] = df["ci_95_lower_x100"].clip(lower=0, upper=100)
    df["ci_95_upper_x100"] = df["ci_95_upper_x100"].clip(lower=0, upper=100)
    
    
    return df

In [ ]:
df = compute_ci(df);
overall_df = compute_ci(overall_df);
country_df = compute_ci(country_df);

In [ ]:
overall_df.head();

# Percentage Severe by Race and Sex, points with error bars, age group dropdown

In [ ]:
chart = alt.Chart(overall_df)

column_width = 105

age_dropdown = alt.binding_select(options=AGE_GROUP_VALUES)
age_selection = alt.selection_single(fields=["age_group"], bind=age_dropdown, name="Age Group", init={"age_group": "All"})

country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values())) 

filtered_chart = chart.transform_filter(
    age_selection
)

tooltip = [
    alt.Tooltip("race", title="Race"),
    alt.Tooltip("sex", title="Sex"),
    alt.Tooltip("age_group", title="Age Group"),
    alt.Tooltip("ci_95_lower_x100", title="Percentage Severe, 95% CI Lower"),
    alt.Tooltip("p_x100", title="Percentage Severe"),
    alt.Tooltip("ci_95_upper_x100", title="Percentage Severe, 95% CI Upper"),
    alt.Tooltip("num_patients_all", title="Number of Patients"),
    alt.Tooltip("num_patients_ever_severe", title="Number of Patients Ever Severe"),
    alt.Tooltip("num_patients_never_severe", title="Number of Patients Never Severe"),
]

top_bar = filtered_chart.mark_point(filled=True, size=90, color="gray").encode(
    x=alt.X("sex:N", axis=alt.Axis(title=None, labels=False, ticks=False)),
    shape=alt.Shape("sex:N", legend=alt.Legend(title="Sex", orient="bottom")),
    y=alt.Y("p_x100:Q", axis=alt.Axis(title="% Severe")),
    tooltip=tooltip
).properties(width=column_width,height=300)


top_errorbar = filtered_chart.mark_errorbar().encode(
    x=alt.X("sex:N", axis=alt.Axis(title="Sex")),
    y=alt.Y("ci_95_upper_x100:Q", title=""), 
    y2=alt.Y2("ci_95_lower_x100:Q", title=""),
    tooltip=tooltip
)

# Create a bottom bar plot to show country counts
bottom_bar = alt.Chart(country_df).transform_filter(
    age_selection
).mark_bar().encode(
    x=alt.X("sex:N", axis=alt.Axis(title="Sex")),
    color=alt.Color("country:N", legend=alt.Legend(title="Country", orient="bottom"), scale=country_color_scale),
    y=alt.Y("num_patients_all:Q", axis=alt.Axis(title="# of patients")),
    tooltip=tooltip + [
        alt.Tooltip("country", title="Country"),
    ]
).properties(width=column_width,height=150)

top_chart = alt.layer(top_bar, top_errorbar).facet(
    column=alt.Column(
        "race:N",
        sort=RACE_VALUES,
        header=alt.Header(title=None)
    ),
)

bottom_chart = bottom_bar.facet(
    column=alt.Column(
        "race:N",
        sort=RACE_VALUES,
        header=alt.Header(title=None, labels=False)
    ),
)



top_and_bottom_chart = alt.vconcat(top_chart, bottom_chart, spacing=10).properties(title={
    "text": ["Percentage Severe by Race and Sex"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

themed_top_and_bottom_chart = apply_theme(top_and_bottom_chart).configure_mark(
    color='orange'
).add_selection(
    age_selection
)

themed_top_and_bottom_chart

# Percentage Severe by Race and Sex, bars with error bars, age group dropdown

In [ ]:
chart = alt.Chart(overall_df)

column_width = 105

age_dropdown = alt.binding_select(options=AGE_GROUP_VALUES)
age_selection = alt.selection_single(fields=["age_group"], bind=age_dropdown, name="Age Group", init={"age_group": "All"})

country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values())) 

filtered_chart = chart.transform_filter(
    age_selection
)

tooltip = [
    alt.Tooltip("race", title="Race"),
    alt.Tooltip("sex", title="Sex"),
    alt.Tooltip("age_group", title="Age Group"),
    alt.Tooltip("ci_95_lower_x100", title="Percentage Severe, 95% CI Lower"),
    alt.Tooltip("p_x100", title="Percentage Severe"),
    alt.Tooltip("ci_95_upper_x100", title="Percentage Severe, 95% CI Upper"),
    alt.Tooltip("num_patients_all", title="Number of Patients"),
    alt.Tooltip("num_patients_ever_severe", title="Number of Patients Ever Severe"),
    alt.Tooltip("num_patients_never_severe", title="Number of Patients Never Severe"),
]

top_bar = filtered_chart.mark_bar(color="silver").encode(
    x=alt.X("sex:N", axis=alt.Axis(title=None, ticks=False, labels=False)),
    y=alt.Y("p_x100:Q", axis=alt.Axis(title="% Severe")),
    tooltip=tooltip
).properties(width=column_width,height=300)


top_errorbar = filtered_chart.mark_errorbar().encode(
    x=alt.X("sex:N", axis=alt.Axis(title="Sex")),
    y=alt.Y("ci_95_upper_x100:Q", title=""), 
    y2=alt.Y2("ci_95_lower_x100:Q", title=""),
    tooltip=tooltip
)

# Create a bottom bar plot to show country counts
bottom_bar = alt.Chart(country_df).transform_filter(
    age_selection
).mark_bar().encode(
    x=alt.X("sex:N", axis=alt.Axis(title="Sex")),
    color=alt.Color("country:N", legend=alt.Legend(title="Country", orient="bottom"), scale=country_color_scale),
    y=alt.Y("num_patients_all:Q", axis=alt.Axis(title="# of patients")),
    tooltip=tooltip + [
        alt.Tooltip("country", title="Country"),
    ]
).properties(width=column_width,height=150)

top_chart = alt.layer(top_bar, top_errorbar).facet(
    column=alt.Column(
        "race:N",
        sort=RACE_VALUES,
        header=alt.Header(title=None)
    ),
)

bottom_chart = bottom_bar.facet(
    column=alt.Column(
        "race:N",
        sort=RACE_VALUES,
        header=alt.Header(title=None, labels=False)
    ),
)



top_and_bottom_chart = alt.vconcat(top_chart, bottom_chart, spacing=10).properties(title={
    "text": ["Percentage Severe by Race and Sex"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

themed_top_and_bottom_chart = apply_theme(top_and_bottom_chart).configure_mark(
    color='orange'
).add_selection(
    age_selection
)

themed_top_and_bottom_chart

In [ ]:
# This is a function to create a plot with 3 levels of stratification:
# variable 1: column facet
# variable 2: x-axis nominal field within each column
# variable 3: dropdown selection to filter the data
def create_plot_var1_colfacet_var2_x_var3_dropdown(var1, var2, var3, column_width=100, tick_size=15):
    
    var1["title"] = HR_COLNAME_MAP[var1["colname"]]
    var2["title"] = HR_COLNAME_MAP[var2["colname"]]
    var3["title"] = HR_COLNAME_MAP[var3["colname"]]
    
    chart = alt.Chart(overall_df)

    var3_dropdown = alt.binding_select(options=var3["values"])
    var3_selection = alt.selection_single(fields=[var3["colname"]], bind=var3_dropdown, name=var3["title"], init={var3["colname"]: "All"})

    country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values()))

    filtered_chart = chart.transform_filter(
        var3_selection
    )

    tooltip = [
        alt.Tooltip("race", title="Race"),
        alt.Tooltip("sex", title="Sex"),
        alt.Tooltip("age_group", title="Age Group"),
        alt.Tooltip("ci_95_lower_x100", title="Percentage Severe, 95% CI Lower"),
        alt.Tooltip("p_x100", title="Percentage Severe"),
        alt.Tooltip("ci_95_upper_x100", title="Percentage Severe, 95% CI Upper"),
        alt.Tooltip("num_patients_all", title="Number of Patients"),
        alt.Tooltip("num_patients_ever_severe", title="Number of Patients Ever Severe"),
        alt.Tooltip("num_patients_never_severe", title="Number of Patients Never Severe"),
    ]

    top_bar = filtered_chart.mark_tick(color="black", size=tick_size, thickness=2).encode(
        x=alt.X(f"{var2['colname']}:{var2['dtype']}", axis=alt.Axis(title=None, ticks=False, labels=False)),
        y=alt.Y("p_x100:Q", axis=alt.Axis(title="% Severe")),
        tooltip=tooltip
    ).properties(width=column_width,height=300)


    top_errorbar = filtered_chart.mark_bar(color="silver", size=tick_size).encode(
        x=alt.X(f"{var2['colname']}:{var2['dtype']}", axis=alt.Axis(title=var2['title'])),
        y=alt.Y("ci_95_upper_x100:Q", title=""), 
        y2=alt.Y2("ci_95_lower_x100:Q", title=""),
        tooltip=tooltip
    ).properties(width=column_width,height=300)

    # Create a bottom bar plot to show country counts
    bottom_bar = alt.Chart(country_df).transform_filter(
        var3_selection
    ).mark_bar().encode(
        x=alt.X(f"{var2['colname']}:{var2['dtype']}", axis=alt.Axis(title=var2['title'])),
        color=alt.Color("country:N", legend=alt.Legend(title="Country", orient="bottom"), scale=country_color_scale),
        y=alt.Y("num_patients_all:Q", axis=alt.Axis(title="# of patients")),
        tooltip=tooltip + [
            alt.Tooltip("country", title="Country"),
        ]
    ).properties(width=column_width,height=150)

    top_chart = alt.layer(top_errorbar, top_bar).facet(
        column=alt.Column(
            f"{var1['colname']}:{var1['dtype']}",
            sort=var1['values'],
            header=alt.Header(title=None)
        ),
    )

    bottom_chart = bottom_bar.facet(
        column=alt.Column(
            f"{var1['colname']}:{var1['dtype']}",
            sort=var1['values'],
            header=alt.Header(title=None, labels=False)
        ),
    )

    top_and_bottom_chart = alt.vconcat(top_chart, bottom_chart, spacing=10).properties(title={
        "text": [f"Percentage Severe by {var1['title']} and {var2['title']}"], 
        "dx": 50,
        "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
        "subtitleColor": "gray",
        "anchor": "middle",
    })

    themed_top_and_bottom_chart = apply_theme(top_and_bottom_chart).configure_mark(
        color='orange'
    ).add_selection(
        var3_selection
    )

    return themed_top_and_bottom_chart

In [ ]:
age_group_plot_var = {
    "colname": "age_group",
    "values": AGE_GROUP_VALUES,
    "dtype": "N"
}

race_plot_var = {
    "colname": "race",
    "values": RACE_VALUES,
    "dtype": "N"
    
}

sex_plot_var = {
    "colname": "sex",
    "values": SEX_VALUES,
    "dtype": "N"
}

In [ ]:
create_plot_var1_colfacet_var2_x_var3_dropdown(age_group_plot_var, race_plot_var, sex_plot_var, 80, 10)

In [ ]:
create_plot_var1_colfacet_var2_x_var3_dropdown(race_plot_var, age_group_plot_var, sex_plot_var, 150, 10)

In [ ]:
create_plot_var1_colfacet_var2_x_var3_dropdown(age_group_plot_var, sex_plot_var, race_plot_var, 85, 12)

In [ ]:
create_plot_var1_colfacet_var2_x_var3_dropdown(sex_plot_var, age_group_plot_var, race_plot_var, 175, 12)

In [ ]:
create_plot_var1_colfacet_var2_x_var3_dropdown(race_plot_var, sex_plot_var, age_group_plot_var, 105, 20)

In [ ]:
create_plot_var1_colfacet_var2_x_var3_dropdown(sex_plot_var, race_plot_var, age_group_plot_var, 175, 25)

# Percentage Severe by Sex and Race, faceted, age group dropdown

In [ ]:
column_width = 100
row_height = 100

age_dropdown = alt.binding_select(options=AGE_GROUP_VALUES)
age_selection = alt.selection_single(fields=["age_group"], bind=age_dropdown, name="Age Group", init={"age_group": "All"})

tooltip = [
    alt.Tooltip("race", title="Race"),
    alt.Tooltip("sex", title="Sex"),
    alt.Tooltip("age_group", title="Age Group"),
    alt.Tooltip("ci_95_lower_x100", title="Percentage Severe, 95% CI Lower"),
    alt.Tooltip("p_x100", title="Percentage Severe"),
    alt.Tooltip("ci_95_upper_x100", title="Percentage Severe, 95% CI Upper"),
    alt.Tooltip("num_patients_all", title="Number of Patients"),
    alt.Tooltip("num_patients_ever_severe", title="Number of Patients Ever Severe"),
    alt.Tooltip("num_patients_never_severe", title="Number of Patients Never Severe"),
]

chart = alt.Chart(overall_df)

filtered_chart = chart.transform_filter(
    age_selection
)

bar = filtered_chart.mark_point(color="gray", filled=True, size=100).encode(
    y=alt.Y("p_x100:Q", axis=alt.Axis(title="% Severe")),
    tooltip=tooltip
).properties(width=column_width,height=row_height)


errorbar = filtered_chart.mark_errorbar(color="black").encode(
    y=alt.Y("ci_95_upper_x100:Q", title=""), 
    y2=alt.Y2("ci_95_lower_x100:Q", title=""),
    tooltip=tooltip
)

plot = alt.layer(bar, errorbar).facet(
    column=alt.Column(
        "race:N",
        sort=RACE_VALUES,
        header=alt.Header(labelOrient="bottom", title="Race", titleOrient="bottom")
    ),
    row=alt.Row(
        "sex:N",
        sort=SEX_VALUES,
        header=alt.Header(title="Sex")
    ),
).properties(title={
    "text": ["Percentage Severe by Sex and Race"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot).add_selection(
    age_selection
)

plot

In [ ]:
column_width = 130
row_height = 110

age_color_scale = alt.Scale(domain=AGE_GROUP_VALUES, range=["#000000", "#C7DFEE", "#BAD8EB", "#AAD0E6", "#9BC8E2", "#8CBEDD", "#7FB5D8", "#73ABD3", "#69A0CD", "#5F96C5", "#D3E5F2"]) 

tooltip = [
    alt.Tooltip("race", title="Race"),
    alt.Tooltip("sex", title="Sex"),
    alt.Tooltip("age_group", title="Age Group"),
    alt.Tooltip("ci_95_lower_x100", title="Percentage Severe, 95% CI Lower"),
    alt.Tooltip("p_x100", title="Percentage Severe"),
    alt.Tooltip("ci_95_upper_x100", title="Percentage Severe, 95% CI Upper"),
    alt.Tooltip("num_patients_all", title="Number of Patients"),
    alt.Tooltip("num_patients_ever_severe", title="Number of Patients Ever Severe"),
    alt.Tooltip("num_patients_never_severe", title="Number of Patients Never Severe"),
]

chart = alt.Chart(overall_df)

bar = chart.mark_point(color="gray", filled=True, size=100).encode(
    x=alt.X("age_group:O", axis=alt.Axis(title=None), sort=AGE_GROUP_VALUES),
    color=alt.Color("age_group:O", legend=alt.Legend(title="Age Group"), sort=AGE_GROUP_VALUES, scale=age_color_scale),
    y=alt.Y("p_x100:Q", axis=alt.Axis(title="% Severe")),
    tooltip=tooltip
).properties(width=column_width,height=row_height)


errorbar = chart.mark_errorbar(color="black").encode(
    x=alt.X("age_group:O", axis=alt.Axis(title="Age Group"), sort=AGE_GROUP_VALUES),
    y=alt.Y("ci_95_upper_x100:Q", title=""), 
    y2=alt.Y2("ci_95_lower_x100:Q", title=""),
    tooltip=tooltip
)

plot = alt.layer(bar, errorbar).facet(
    column=alt.Column(
        "race:N",
        sort=RACE_VALUES,
        header=alt.Header(labelOrient="top", title=None, titleOrient="top")
    ),
    row=alt.Row(
        "sex:N",
        sort=SEX_VALUES,
        header=alt.Header(title="Sex")
    ),
).properties(title={
    "text": ["Percentage Severe by Sex, Race, and Age Group"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

plot

In [ ]:
column_width = 130
row_height = 110

age_color_scale = alt.Scale(domain=AGE_GROUP_VALUES, range=["#000000", "#C7DFEE", "#BAD8EB", "#AAD0E6", "#9BC8E2", "#8CBEDD", "#7FB5D8", "#73ABD3", "#69A0CD", "#5F96C5", "#D3E5F2"]) 

tooltip = [
    alt.Tooltip("race", title="Race"),
    alt.Tooltip("sex", title="Sex"),
    alt.Tooltip("age_group", title="Age Group"),
    alt.Tooltip("ci_95_lower_x100", title="Percentage Severe, 95% CI Lower"),
    alt.Tooltip("p_x100", title="Percentage Severe"),
    alt.Tooltip("ci_95_upper_x100", title="Percentage Severe, 95% CI Upper"),
    alt.Tooltip("num_patients_all", title="Number of Patients"),
    alt.Tooltip("num_patients_ever_severe", title="Number of Patients Ever Severe"),
    alt.Tooltip("num_patients_never_severe", title="Number of Patients Never Severe"),
]

chart = alt.Chart(overall_df)

bar = chart.mark_point(color="gray", filled=True, size=100).encode(
    x=alt.X("age_group:O", axis=alt.Axis(title="Age Group"), sort=AGE_GROUP_VALUES),
    shape=alt.Shape("sex:N", legend=alt.Legend(title="Sex")),
    color=alt.Color("age_group:O", legend=alt.Legend(title="Age Group"), sort=AGE_GROUP_VALUES, scale=age_color_scale),
    y=alt.Y("p_x100:Q", axis=alt.Axis(title="% Severe")),
    tooltip=tooltip
).properties(width=column_width,height=row_height)


errorbar = chart.mark_errorbar(color="black").encode(
    x=alt.X("age_group:O", axis=alt.Axis(title="Age Group"), sort=AGE_GROUP_VALUES),
    y=alt.Y("ci_95_upper_x100:Q", title=""), 
    y2=alt.Y2("ci_95_lower_x100:Q", title=""),
    tooltip=tooltip
)

plot = alt.layer(bar, errorbar).facet(
    column=alt.Column(
        "race:N",
        sort=RACE_VALUES,
        header=alt.Header(labelOrient="top", title=None, titleOrient="bottom")
    ),
    row=alt.Row(
        "sex:N",
        sort=SEX_VALUES,
        header=alt.Header(title="Sex")
    ),
).properties(title={
    "text": ["Percentage Severe by Sex, Race, and Age Group"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

plot

In [ ]:
column_width = 130
row_height = 110

age_color_scale = alt.Scale(domain=AGE_GROUP_VALUES, range=["#000000", "#C7DFEE", "#BAD8EB", "#AAD0E6", "#9BC8E2", "#8CBEDD", "#7FB5D8", "#73ABD3", "#69A0CD", "#5F96C5", "#D3E5F2"]) 

tooltip = [
    alt.Tooltip("race", title="Race"),
    alt.Tooltip("sex", title="Sex"),
    alt.Tooltip("age_group", title="Age Group"),
    alt.Tooltip("ci_95_lower_x100", title="Percentage Severe, 95% CI Lower"),
    alt.Tooltip("p_x100", title="Percentage Severe"),
    alt.Tooltip("ci_95_upper_x100", title="Percentage Severe, 95% CI Upper"),
    alt.Tooltip("num_patients_all", title="Number of Patients"),
    alt.Tooltip("num_patients_ever_severe", title="Number of Patients Ever Severe"),
    alt.Tooltip("num_patients_never_severe", title="Number of Patients Never Severe"),
]

chart = alt.Chart(overall_df)

bar = chart.mark_point(color="gray", filled=True, size=100).encode(
    x=alt.X("age_group:O", axis=alt.Axis(title="Age Group"), sort=AGE_GROUP_VALUES),
    shape=alt.Shape("race:N", legend=alt.Legend(title="Race")),
    color=alt.Color("age_group:O", legend=alt.Legend(title="Age Group"), sort=AGE_GROUP_VALUES, scale=age_color_scale),
    y=alt.Y("p_x100:Q", axis=alt.Axis(title="% Severe")),
    tooltip=tooltip
).properties(width=column_width,height=row_height)


errorbar = chart.mark_errorbar(color="black").encode(
    x=alt.X("age_group:O", axis=alt.Axis(title="Age Group"), sort=AGE_GROUP_VALUES),
    y=alt.Y("ci_95_upper_x100:Q", title=""), 
    y2=alt.Y2("ci_95_lower_x100:Q", title=""),
    tooltip=tooltip
)

plot = alt.layer(bar, errorbar).facet(
    column=alt.Column(
        "race:N",
        sort=RACE_VALUES,
        header=alt.Header(labelOrient="top", title=None, titleOrient="bottom")
    ),
    row=alt.Row(
        "sex:N",
        sort=SEX_VALUES,
        header=alt.Header(title="Sex")
    ),
).properties(title={
    "text": ["Percentage Severe by Sex, Race, and Age Group"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

plot

# Count by severity and age group, bar plot, country dropdown

In [ ]:
by_age_group_and_country_df = country_df.loc[(country_df["race"] == "All") & (country_df["sex"] == "All")]
by_age_group_and_country_df.head();

In [ ]:
by_age_group_agg_df = by_age_group_and_country_df.groupby(["age_group", "sex", "race"]).sum().reset_index()
by_age_group_agg_df["country"] = "All"
by_age_group_agg_df["color"] = np.nan
by_age_group_agg_df = compute_ci(by_age_group_agg_df)
by_age_group_agg_df;

In [ ]:
by_age_group_and_country_df = by_age_group_and_country_df.append(by_age_group_agg_df, ignore_index=True)
by_age_group_and_country_df.tail();

In [ ]:
by_age_group_and_country_molten_df = by_age_group_and_country_df.melt(id_vars=set(by_age_group_and_country_df.columns.values.tolist()) - set(["num_patients_all", "num_patients_ever_severe", "num_patients_never_severe"]))
by_age_group_and_country_molten_df = by_age_group_and_country_molten_df.loc[by_age_group_and_country_molten_df["variable"] != "num_patients_all"]
by_age_group_and_country_molten_df = by_age_group_and_country_molten_df.loc[~by_age_group_and_country_molten_df["age_group"].isin(["All"])]
by_age_group_and_country_molten_df["variable"] = by_age_group_and_country_molten_df["variable"].replace(HR_SEVERITY_COLNAME_MAP)
by_age_group_and_country_molten_df;

In [ ]:
column_width = 60

country_dropdown = alt.binding_select(options=["All"] + COUNTRY_VALUES)
country_selection = alt.selection_single(fields=["country"], bind=country_dropdown, name="Country", init={"country": "All"})

tooltip = [
    alt.Tooltip("age_group", title="Age Group"),
    alt.Tooltip("variable", title="Severity"),
    alt.Tooltip("value", title="Number of Patients"),
]

chart = alt.Chart(by_age_group_and_country_molten_df)

filtered_chart = chart.transform_filter(
    country_selection
)

bar = filtered_chart.mark_bar().encode(
    y=alt.Y("value:Q", axis=alt.Axis(title="Number of Patients")),
    x=alt.X("variable:O", axis=alt.Axis(title="Severity"), sort=["Never Severe", "Ever Severe"]),
    color=alt.X("variable:O", legend=alt.Legend(title="Severity", orient="top-left"), sort=["Never Severe", "Ever Severe"]),
    tooltip=tooltip
).properties(width=column_width,height=300)


plot = bar.facet(
    column=alt.Column(
        "age_group:N",
        sort=AGE_GROUP_VALUES,
        header=alt.Header(labelOrient="bottom", title="Age Group", titleOrient="bottom")
    ),
).properties(title={
    "text": ["Number of Patients by Age Group and Severity"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot).add_selection(
    country_selection
)

plot

In [ ]:
column_width = 60

by_age_group_and_country_molten_without_all_df = by_age_group_and_country_molten_df.loc[by_age_group_and_country_molten_df["country"] != "All"]

country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values()))

tooltip = [
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("age_group", title="Age Group"),
    alt.Tooltip("variable", title="Severity"),
    alt.Tooltip("value", title="Number of Patients"),
]

chart = alt.Chart(by_age_group_and_country_molten_without_all_df)

bar = chart.mark_bar().encode(
    y=alt.Y("value:Q", axis=alt.Axis(title="Number of Patients")),
    x=alt.X("variable:O", axis=alt.Axis(title="Severity"), sort=["Never Severe", "Ever Severe"]),
    color=alt.X("country:N", legend=alt.Legend(title="Country", orient="top-left"), sort=COUNTRY_VALUES, scale=country_color_scale),
    tooltip=tooltip
).properties(width=column_width,height=300)


plot = bar.facet(
    column=alt.Column(
        "age_group:N",
        sort=AGE_GROUP_VALUES,
        header=alt.Header(labelOrient="bottom", title="Age Group", titleOrient="bottom")
    ),
).properties(title={
    "text": ["Number of Patients by Age Group, Country, and Severity"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

plot

In [ ]:
column_width = 60

by_age_group_and_country_molten_without_all_df = by_age_group_and_country_molten_df.loc[by_age_group_and_country_molten_df["country"] != "All"]

country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values()))

tooltip = [
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("age_group", title="Age Group"),
    alt.Tooltip("variable", title="Severity"),
    alt.Tooltip("value", title="Number of Patients"),
]

chart = alt.Chart(by_age_group_and_country_molten_without_all_df)

bar = chart.mark_bar().encode(
    y=alt.Y("value:Q", axis=alt.Axis(title="Number of Patients")),
    x=alt.X("variable:O", axis=alt.Axis(title="Severity"), sort=["Never Severe", "Ever Severe"]),
    opacity=alt.X("variable:N", legend=alt.Legend(title="Severity", orient="top-left"), sort=["Never Severe", "Ever Severe"]),
    color=alt.X("country:N", legend=alt.Legend(title="Country", orient="top-left"), sort=COUNTRY_VALUES, scale=country_color_scale),
    tooltip=tooltip
).properties(width=column_width,height=300)


plot = bar.facet(
    column=alt.Column(
        "age_group:N",
        sort=AGE_GROUP_VALUES,
        header=alt.Header(labelOrient="bottom", title="Age Group", titleOrient="bottom")
    ),
).properties(title={
    "text": ["Number of Patients by Age Group, Country, and Severity"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

plot

# Count by severity and race, bar plot, country dropdown

In [ ]:
by_race_and_country_df = country_df.loc[(country_df["age_group"] == "All") & (country_df["sex"] == "All")]
by_race_and_country_df.head();

In [ ]:
by_race_agg_df = by_race_and_country_df.groupby(["age_group", "sex", "race"]).sum().reset_index()
by_race_agg_df["country"] = "All"
by_race_agg_df["color"] = np.nan
by_race_agg_df = compute_ci(by_race_agg_df)
by_race_agg_df;

In [ ]:
by_race_and_country_df = by_race_and_country_df.append(by_race_agg_df, ignore_index=True)
by_race_and_country_df.tail();

In [ ]:
by_race_and_country_molten_df = by_race_and_country_df.melt(id_vars=set(by_race_and_country_df.columns.values.tolist()) - set(["num_patients_all", "num_patients_ever_severe", "num_patients_never_severe"]))
by_race_and_country_molten_df = by_race_and_country_molten_df.loc[by_race_and_country_molten_df["variable"] != "num_patients_all"]
by_race_and_country_molten_df = by_race_and_country_molten_df.loc[~by_race_and_country_molten_df["race"].isin(["All"])]
by_race_and_country_molten_df["variable"] = by_race_and_country_molten_df["variable"].replace(HR_SEVERITY_COLNAME_MAP)
by_race_and_country_molten_df;

In [ ]:
column_width = 60

country_dropdown = alt.binding_select(options=["All"] + COUNTRY_VALUES)
country_selection = alt.selection_single(fields=["country"], bind=country_dropdown, name="Country", init={"country": "All"})

tooltip = [
    alt.Tooltip("race", title="Race"),
    alt.Tooltip("variable", title="Severity"),
    alt.Tooltip("value", title="Number of Patients"),
]

chart = alt.Chart(by_race_and_country_molten_df)

filtered_chart = chart.transform_filter(
    country_selection
)

bar = filtered_chart.mark_bar().encode(
    y=alt.Y("value:Q", axis=alt.Axis(title="Number of Patients")),
    x=alt.X("variable:O", axis=alt.Axis(title="Severity"), sort=["Never Severe", "Ever Severe"]),
    color=alt.X("variable:O", legend=alt.Legend(title="Severity", orient="right"), sort=["Never Severe", "Ever Severe"]),
    tooltip=tooltip
).properties(width=column_width,height=300)


plot = bar.facet(
    column=alt.Column(
        "race:N",
        sort=RACE_VALUES,
        header=alt.Header(labelOrient="bottom", title="Race", titleOrient="bottom")
    ),
).properties(title={
    "text": ["Number of Patients by Race and Severity"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot).add_selection(
    country_selection
)

plot

# Count by severity and sex, bar plot, country dropdown

In [ ]:
by_sex_and_country_df = country_df.loc[(country_df["race"] == "All") & (country_df["age_group"] == "All")]
by_sex_and_country_df.head();

In [ ]:
by_sex_agg_df = by_sex_and_country_df.groupby(["age_group", "sex", "race"]).sum().reset_index()
by_sex_agg_df["country"] = "All"
by_sex_agg_df["color"] = np.nan
by_sex_agg_df = compute_ci(by_sex_agg_df)
by_sex_agg_df;

In [ ]:
by_sex_and_country_df = by_sex_and_country_df.append(by_sex_agg_df, ignore_index=True)
by_sex_and_country_df.tail();

In [ ]:
by_sex_and_country_molten_df = by_sex_and_country_df.melt(id_vars=set(by_sex_and_country_df.columns.values.tolist()) - set(["num_patients_all", "num_patients_ever_severe", "num_patients_never_severe"]))
by_sex_and_country_molten_df = by_sex_and_country_molten_df.loc[by_sex_and_country_molten_df["variable"] != "num_patients_all"]
by_sex_and_country_molten_df = by_sex_and_country_molten_df.loc[~by_sex_and_country_molten_df["sex"].isin(["All"])]
by_sex_and_country_molten_df["variable"] = by_sex_and_country_molten_df["variable"].replace(HR_SEVERITY_COLNAME_MAP)
by_sex_and_country_molten_df;

In [ ]:
column_width = 60

country_dropdown = alt.binding_select(options=["All"] + COUNTRY_VALUES)
country_selection = alt.selection_single(fields=["country"], bind=country_dropdown, name="Country", init={"country": "All"})

tooltip = [
    alt.Tooltip("sex", title="Sex"),
    alt.Tooltip("variable", title="Severity"),
    alt.Tooltip("value", title="Number of Patients"),
]

chart = alt.Chart(by_sex_and_country_molten_df)

filtered_chart = chart.transform_filter(
    country_selection
)

bar = filtered_chart.mark_bar().encode(
    y=alt.Y("value:Q", axis=alt.Axis(title="Number of Patients")),
    x=alt.X("variable:O", axis=alt.Axis(title="Severity"), sort=["Never Severe", "Ever Severe"]),
    color=alt.X("variable:O", legend=alt.Legend(title="Severity", orient="right"), sort=["Never Severe", "Ever Severe"]),
    tooltip=tooltip
).properties(width=column_width,height=300)


plot = bar.facet(
    column=alt.Column(
        "sex:N",
        sort=SEX_VALUES,
        header=alt.Header(labelOrient="bottom", title="Sex", titleOrient="bottom")
    ),
).properties(title={
    "text": ["Number of Patients by Sex and Severity"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot).add_selection(
    country_selection
)

plot

# Demographics plots from phase 1.0

Similar to https://covidclinical.net/plots/paper-01/release-2020-04-11/demographics.html from phase 1.0

In [ ]:
def get_df_by_country_and_severity(*include_vars, with_all_countries=True, keep_all_overrides=[], ci_group_by=None):
    all_vars = ["age_group", "sex", "race"]
    exclude_vars = list(set(all_vars) - set(include_vars) - set(keep_all_overrides))
    
    # Only keep the "All" rows for the variables we want to ignore.
    by_vars_and_country_df = country_df
    for e_var in exclude_vars:
        by_vars_and_country_df = by_vars_and_country_df.loc[(country_df[e_var] == "All")]
    
    # Do the "All"-country aggregation.
    if with_all_countries:
        by_vars_agg_df = by_vars_and_country_df.groupby(all_vars).sum().reset_index()
        by_vars_agg_df["country"] = "All"
        by_vars_agg_df["color"] = np.nan

        by_vars_and_country_df = by_vars_and_country_df.append(by_vars_agg_df, ignore_index=True)
   
    by_vars_and_country_molten_df = by_vars_and_country_df.melt(id_vars=set(by_vars_and_country_df.columns.values.tolist()) - set(["num_patients_all", "num_patients_ever_severe", "num_patients_never_severe"]))
    by_vars_and_country_molten_df = by_vars_and_country_molten_df.loc[by_vars_and_country_molten_df["variable"] != "num_patients_all"]
    
    # Remove the "All" rows for the variables of interest.
    for i_var in include_vars:
        if i_var not in keep_all_overrides:
            by_vars_and_country_molten_df = by_vars_and_country_molten_df.loc[~by_vars_and_country_molten_df[i_var].isin(["All"])]
    
    by_vars_and_country_molten_df["variable"] = by_vars_and_country_molten_df["variable"].replace(HR_SEVERITY_COLNAME_MAP)
    
    # Compute confidence intervals
    if ci_group_by != None:
        ci_all_colname = ci_group_by + "_n"
        ci_df = pd.DataFrame()

        for index_val, country_severity_df in by_vars_and_country_molten_df.groupby(["country", ci_group_by, "variable"]):
            country_n = country_severity_df.sum()["value"] # total number of cases in the country AND in this severity category
            country_severity_df[ci_all_colname] = country_n

            ci_df = ci_df.append(country_severity_df, ignore_index=True)

        ci_df = compute_ci(ci_df, n_col="value", N_col=ci_all_colname)
        return ci_df
    
    return by_vars_and_country_molten_df

For the confidence intervals:
- For percentages in demographics
    - $P = \texttt{percentage of patients in group in country}$ (for example age group 50-69 in France, or 50-69 AND Male in France)
    - $N = \texttt{total number of cases in country}$
    - then the standard error should be $P*(1-P)/N$
    - so 95% CI would be $P \pm 1.96*\sqrt{P*(1-P)/N}$


In [ ]:
by_age_group_and_country_molten_df = get_df_by_country_and_severity("age_group")
by_age_group_and_country_molten_df.head()

ci_df = pd.DataFrame()

for (country, severity), country_severity_df in by_age_group_and_country_molten_df.groupby(["country", "variable"]):
    country_n = country_severity_df.sum()["value"] # total number of cases in the country AND in this severity category
    country_severity_df["country_n"] = country_n
    
    ci_df = ci_df.append(country_severity_df, ignore_index=True)

ci_df = compute_ci(ci_df, n_col="value", N_col="country_n")
ci_df.loc[(ci_df["country"] == "USA") & (ci_df["variable"] == "Ever Severe")]

In [ ]:
country_df.loc[(country_df["country"] == "USA") & (country_df["age_group"] == "0 - 2") & (country_df["race"] == "All") & (country_df["sex"])]

In [ ]:
# This is a function to create a plot with 2 dynamic variables:
# variable 1: column facet
# variable 2: x-axis nominal field within each column
# country dropdown selection to filter the data
# severity row facet
def create_plot_var1_colfacet_var2_x_and_severity_and_country(var1, var2, column_width=60, row_height=200):
    
    var1["title"] = HR_COLNAME_MAP[var1["colname"]]
    var2["title"] = HR_COLNAME_MAP[var2["colname"]]

    by_var1_var2_and_country_molten_df = get_df_by_country_and_severity(var1["colname"], var2["colname"])

    country_dropdown = alt.binding_select(options=["All"] + COUNTRY_VALUES)
    country_selection = alt.selection_single(fields=["country"], bind=country_dropdown, name="Country", init={"country": "All"})

    tooltip = [
        alt.Tooltip(var1["colname"], title=var1["title"]),
        alt.Tooltip(var2['colname'], title=var2['title']),
        alt.Tooltip("country", title="Country"),
        alt.Tooltip("variable", title="Severity"),
        alt.Tooltip("value", title="Number of Patients"),
    ]

    chart = alt.Chart(by_var1_var2_and_country_molten_df)

    filtered_chart = chart.transform_filter(
        country_selection
    )

    bar = filtered_chart.mark_bar().encode(
        y=alt.Y("value:Q", axis=alt.Axis(title="Number of Patients")),
        x=alt.X(f"{var2['colname']}:{var2['dtype']}", axis=alt.Axis(title=var2['title']), sort=var2['values']),
        color=alt.Color(f"{var2['colname']}:{var2['dtype']}", legend=alt.Legend(title=None, orient="right", labelLimit=300), sort=var2['values']),
        tooltip=tooltip
    ).properties(width=column_width, height=row_height)


    plot = bar.facet(
        column=alt.Column(
            f"{var1['colname']}:{var1['dtype']}",
            sort=var1["values"],
            header=alt.Header(labelOrient="bottom", title=var1["title"], titleOrient="bottom")
        ),
        row=alt.Row(
            "variable:O",
            sort=["Never Severe", "Ever Severe"],
            header=alt.Header(labelOrient="left", title="Severity", titleOrient="left")
        ),
    ).properties(title={
        "text": [f"Number of Patients by {var2['title']} and {var1['title']}"], 
        "dx": 50,
        "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
        "subtitleColor": "gray",
        "anchor": "middle",
    })

    plot = apply_theme(plot).add_selection(
        country_selection
    )
    
    for_website(plot, f"Demographics {COHORT}", f"plot_{var1['colname']}_colfacet_{var2['colname']}_x_and_severity_and_country")

    return plot

In [ ]:
create_plot_var1_colfacet_var2_x_and_severity_and_country(age_group_plot_var, sex_plot_var, 60, 200)

In [ ]:
create_plot_var1_colfacet_var2_x_and_severity_and_country(age_group_plot_var, race_plot_var, 100, 200)

# Demographics by Country

Similar to https://covidclinical.net/plots/paper-01/release-2020-04-11/demographics.html?tab=1 from phase 1.0

In [ ]:
# This is a function to create a plot with 2 dynamic variables:
# variable 1: column facet
# variable 2: dropdown selection to filter the data
# country as x-axis nominal color field
# severity as row facet
def create_plot_var1_colfacet_var2_dropdown_and_severity_and_country(var1, var2, column_width=100, row_height=200):
    
    var1["title"] = HR_COLNAME_MAP[var1["colname"]]
    var2["title"] = HR_COLNAME_MAP[var2["colname"]]

    by_var1_var2_and_country_molten_df = get_df_by_country_and_severity(var1["colname"], with_all_countries=False, keep_all_overrides=[var2['colname']])

    var2_dropdown = alt.binding_select(options=var2['values'])
    var2_selection = alt.selection_single(fields=[var2['colname']], bind=var2_dropdown, name=var2['title'], init={var2['colname']: "All"})

    tooltip = [
        alt.Tooltip(var1["colname"], title=var1["title"]),
        alt.Tooltip(var2['colname'], title=var2['title']),
        alt.Tooltip("country", title="Country"),
        alt.Tooltip("variable", title="Severity"),
        alt.Tooltip("value", title="Number of Patients"),
    ]

    chart = alt.Chart(by_var1_var2_and_country_molten_df)

    filtered_chart = chart.transform_filter(
        var2_selection
    )

    bar = filtered_chart.mark_bar().encode(
        y=alt.Y("value:Q", axis=alt.Axis(title="Number of Patients")),
        x=alt.X("country:N", axis=alt.Axis(title="Country"), sort=COUNTRY_VALUES),
        color=alt.Color("country:N", legend=alt.Legend(title=None, orient="right", labelLimit=300), sort=COUNTRY_VALUES),
        tooltip=tooltip
    ).properties(width=column_width, height=row_height)


    plot = bar.facet(
        column=alt.Column(
            f"{var1['colname']}:{var1['dtype']}",
            sort=var1["values"],
            header=alt.Header(labelOrient="bottom", title=var1["title"], titleOrient="bottom")
        ),
        row=alt.Row(
            "variable:O",
            sort=["Never Severe", "Ever Severe"],
            header=alt.Header(labelOrient="left", title="Severity", titleOrient="left")
        ),
    ).properties(title={
        "text": [f"Number of Patients by {var1['title']} and Country"], 
        "dx": 50,
        "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
        "subtitleColor": "gray",
        "anchor": "middle",
    })

    plot = apply_theme(plot).add_selection(
        var2_selection
    )
    
    for_website(plot, f"Demographics {COHORT}", f"plot_{var1['colname']}_colfacet_{var2['colname']}_dropdown_and_severity_and_country")

    return plot

In [ ]:
create_plot_var1_colfacet_var2_dropdown_and_severity_and_country(age_group_plot_var, sex_plot_var, 100, 200)

In [ ]:
create_plot_var1_colfacet_var2_dropdown_and_severity_and_country(age_group_plot_var, race_plot_var, 100, 200)

# With percentages

In [ ]:
# This is a function to create a plot with 2 dynamic variables:
# variable 1: column facet
# variable 2: x-axis nominal field within each column
# country dropdown selection to filter the data
# severity row facet
def create_percentage_plot_var1_colfacet_var2_x_and_severity_and_country(var1, var2, column_width=60, row_height=200):
    
    var1["title"] = HR_COLNAME_MAP[var1["colname"]]
    var2["title"] = HR_COLNAME_MAP[var2["colname"]]

    by_var1_var2_and_country_molten_df = get_df_by_country_and_severity(var1["colname"], var2["colname"], ci_group_by=var2['colname'])

    country_dropdown = alt.binding_select(options=["All"] + COUNTRY_VALUES)
    country_selection = alt.selection_single(fields=["country"], bind=country_dropdown, name="Country", init={"country": "All"})

    tooltip = [
        alt.Tooltip(var1["colname"], title=var1["title"]),
        alt.Tooltip(var2['colname'], title=var2['title']),
        alt.Tooltip("country", title="Country"),
        alt.Tooltip("variable", title="Severity"),
        alt.Tooltip("value", title="Number of Patients"),
        alt.Tooltip("ci_95_lower_x100", title="Percentage of Patients, 95% CI Lower"),
        alt.Tooltip("p_x100", title="Percentage of Patients"),
        alt.Tooltip("ci_95_upper_x100", title="Percentage of Patients, 95% CI Upper"),
    ]

    chart = alt.Chart(by_var1_var2_and_country_molten_df)

    filtered_chart = chart.transform_filter(
        country_selection
    )

    bar = filtered_chart.mark_bar().encode(
        y=alt.Y("p_x100:Q", axis=alt.Axis(title="Percentage of Patients (%)")),
        x=alt.X(f"{var2['colname']}:{var2['dtype']}", axis=alt.Axis(title=var2['title']), sort=var2['values']),
        color=alt.Color(f"{var2['colname']}:{var2['dtype']}", legend=alt.Legend(title=None, orient="right", labelLimit=300), sort=var2['values']),
        tooltip=tooltip
    ).properties(width=column_width, height=row_height)


    errorbar = filtered_chart.mark_errorbar().encode(
        x=alt.X(f"{var2['colname']}:{var2['dtype']}", axis=alt.Axis(title=var2['title']), sort=var2['values']),
        y=alt.Y("ci_95_upper_x100:Q", title=""), 
        y2=alt.Y2("ci_95_lower_x100:Q", title=""),
        tooltip=tooltip
    )


    plot = alt.layer(bar, errorbar).facet(
        column=alt.Column(
            f"{var1['colname']}:{var1['dtype']}",
            sort=var1["values"],
            header=alt.Header(labelOrient="bottom", title=var1["title"], titleOrient="bottom")
        ),
        row=alt.Row(
            "variable:O",
            sort=["Never Severe", "Ever Severe"],
            header=alt.Header(labelOrient="left", title="Severity", titleOrient="left")
        ),
    ).properties(title={
        "text": [f"Percentage of Patients by {var1['title']} within Country, Severity, and {var2['title']}"], 
        "dx": 50,
        "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
        "subtitleColor": "gray",
        "anchor": "middle",
    })

    plot = apply_theme(plot).add_selection(
        country_selection
    )
    
    for_website(plot, f"Demographics {COHORT}", f"percentage_plot_{var1['colname']}_colfacet_{var2['colname']}_x_and_severity_and_country")

    return plot

In [ ]:
create_percentage_plot_var1_colfacet_var2_x_and_severity_and_country(age_group_plot_var, sex_plot_var, 60, 200)#.save('percentage_plot_age_colfacet_sex_x_and_severity_and_country.html')

In [ ]:
create_percentage_plot_var1_colfacet_var2_x_and_severity_and_country(age_group_plot_var, race_plot_var, 100, 200)#.save('percentage_plot_age_colfacet_race_x_and_severity_and_country.html')

In [ ]:
# This is a function to create a plot with 2 dynamic variables:
# variable 1: column facet
# variable 2: dropdown selection to filter the data
# country as x-axis nominal color field
# severity as row facet
def create_percentage_plot_var1_colfacet_var2_dropdown_and_severity_and_country(var1, var2, column_width=100, row_height=200):
    
    var1["title"] = HR_COLNAME_MAP[var1["colname"]]
    var2["title"] = HR_COLNAME_MAP[var2["colname"]]

    by_var1_var2_and_country_molten_df = get_df_by_country_and_severity(var1["colname"], with_all_countries=False, keep_all_overrides=[var2['colname']], ci_group_by=var2["colname"])

    var2_dropdown = alt.binding_select(options=var2['values'])
    var2_selection = alt.selection_single(fields=[var2['colname']], bind=var2_dropdown, name=var2['title'], init={var2['colname']: "All"})

    tooltip = [
        alt.Tooltip(var1["colname"], title=var1["title"]),
        alt.Tooltip(var2['colname'], title=var2['title']),
        alt.Tooltip("country", title="Country"),
        alt.Tooltip("variable", title="Severity"),
        alt.Tooltip("value", title="Number of Patients"),
        alt.Tooltip("ci_95_lower_x100", title="Percentage of Patients, 95% CI Lower"),
        alt.Tooltip("p_x100", title="Percentage of Patients"),
        alt.Tooltip("ci_95_upper_x100", title="Percentage of Patients, 95% CI Upper"),
    ]

    chart = alt.Chart(by_var1_var2_and_country_molten_df)

    filtered_chart = chart.transform_filter(
        var2_selection
    )

    bar = filtered_chart.mark_bar().encode(
        y=alt.Y("p_x100:Q", axis=alt.Axis(title="Percentage of Patients (%)")),
        x=alt.X("country:N", axis=alt.Axis(title="Country"), sort=COUNTRY_VALUES),
        color=alt.Color("country:N", legend=alt.Legend(title=None, orient="right", labelLimit=300), sort=COUNTRY_VALUES),
        tooltip=tooltip
    ).properties(width=column_width, height=row_height)
    
    errorbar = filtered_chart.mark_errorbar().encode(
        x=alt.X("country:N", axis=alt.Axis(title=var2['title']), sort=var2['values']),
        y=alt.Y("ci_95_upper_x100:Q", title=""), 
        y2=alt.Y2("ci_95_lower_x100:Q", title=""),
        tooltip=tooltip
    )


    plot = alt.layer(bar, errorbar).facet(
        column=alt.Column(
            f"{var1['colname']}:{var1['dtype']}",
            sort=var1["values"],
            header=alt.Header(labelOrient="bottom", title=var1["title"], titleOrient="bottom")
        ),
        row=alt.Row(
            "variable:O",
            sort=["Never Severe", "Ever Severe"],
            header=alt.Header(labelOrient="left", title="Severity", titleOrient="left")
        ),
    ).properties(title={
        "text": [f"Percentage of Patients by {var1['title']} within Country, Severity, and {var2['title']}"], 
        "dx": 50,
        "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, cohort=COHORT, num_sites=NUM_SITES),
        "subtitleColor": "gray",
        "anchor": "middle",
    })

    plot = apply_theme(plot).add_selection(
        var2_selection
    )
    
    for_website(plot, f"Demographics {COHORT}", f"percentage_plot_{var1['colname']}_colfacet_{var2['colname']}_dropdown_and_severity_and_country")

    return plot

In [ ]:
create_percentage_plot_var1_colfacet_var2_dropdown_and_severity_and_country(age_group_plot_var, sex_plot_var, 100, 200)#.save('percentage_plot_age_colfacet_sex_dropdown_and_severity_and_country.html')

In [ ]:
create_percentage_plot_var1_colfacet_var2_dropdown_and_severity_and_country(age_group_plot_var, race_plot_var, 100, 200)#.save('percentage_plot_age_colfacet_race_dropdown_and_severity_and_country.html')